In [1]:
import pandas as pd
import numpy as np
import spacy
import csv
import pickle
import collections
from tqdm import tqdm
from math import log10
from tqdm._tqdm_notebook import tqdm_notebook

import textacy

In [3]:
##GET TFIDF TO DICTIONARY
df = pd.read_csv("/Users/boom/Desktop/NewTSP/Data/processedTrain.csv")

counter = collections.Counter()
for index,doc in tqdm(df.iterrows()):
    doc1  = str(doc["q1"]).split(" ")
    doc2 = str(doc["q2"]).split(" ")
    counter.update(doc1)
    counter.update(doc2)
    
def term_frequency(sentence,counter):
    tfCounter = collections.Counter()
    out = []
    sent = str(sentence).split(" ")
    tfCounter.update(sent)
    for w in sent:
        tf = (float(tfCounter.get(w))/float(counter.get(w)))
        out.append((w,tf))
    tfCounter.clear()
    return out


dic = {}
for i in tqdm(counter):
    dic[i] =0

    
for index,doc in tqdm(df.iterrows()):
    doc1  = str(doc["q1"]).split(" ")
    doc2 = str(doc["q2"]).split(" ")

    for word in set(doc1):
        dic[word]+=1
        
    for word in set(doc2):
        dic[word]+=1
    
    
totalDoc = len(df)*2


logIDF = {}
for w in dic:
    logIDF[w] = log10(float(totalDoc)/dic[w])
        
        
tfidf = set()

for index,doc in tqdm(df.iterrows()):
    doc1  = str(doc["q1"]).split(" ")
    doc2 = str(doc["q2"]).split(" ")
    
    sent = []
    for w in term_frequency(doc["q1"],counter):
        tf = (w[1])
        idf = (logIDF[w[0]])
        temp = (w[0],tf*idf)
        tfidf.add(temp)

    
    for w in term_frequency(doc["q2"],counter):
        tf = (w[1])
        idf = (logIDF[w[0]])
        temp = (w[0],tf*idf)
        tfidf.add(temp)

        
tfidf = dict(tfidf)
    
    


404290it [00:56, 7094.60it/s]
100%|██████████| 79296/79296 [00:00<00:00, 1150266.58it/s]
404290it [00:54, 7447.32it/s]
404290it [01:34, 4290.81it/s]


In [4]:
class Glove:
    def __init__(self,modelPath,tk,tfidf = None):
        self.model = model
        self.tk = tk
        self.tfidf = tfidf
        self.progress = 0
    def getVec(self,w):
        return self.model.loc[w].as_matrix()
    def getVectorSet(self,sent):
        t = self.tk(sent )
        vecSet = []
        for word in t:
            try:
                vecTemp = self.getVec(word.text)
                vecSet.append(vecTemp)
            except:
                pass

        return vecSet
    
    def getAverageVector(self, sent):
        vecSet = self.getVectorSet(sent)
        if(len(vecSet)==0):
            return np.zeros(300)
        totalVec = np.zeros(len(vecSet[0]))
        for vec in vecSet:
            totalVec +=vec
        return totalVec/len(vecSet)
    
    def getVectorSetWeighted(self,sent):
        t = self.tk(sent)
        vecSet = []
        total = self.getTotalTFIDF(sent)
        for word in t:
            try:
                vecTemp = self.getVec(word.text)
                vecTemp = vecTemp * (tfidf[word.text]/total)
                vecSet.append(vecTemp)
            except:
                pass
        return vecSet
    
    def getTotalTFIDF(self,sent):
        t = self.tk(sent)
        totalTFIDF = 0
        for word in t:
            try:
                totalTFIDF += tfidf[word.text]
            except:
                pass
        return totalTFIDF
    
    
    def getAverageTFIDF(self, sent):
        vecSet = self.getVectorSetWeighted(sent)
        if(len(vecSet)==0):
            return np.zeros(300)
        totalVec = np.zeros(len(vecSet[0]))
        for vec in vecSet:
            totalVec +=vec
        return totalVec/len(vecSet)
    
    
def CosineSimilarity(vec1,vec2):
    try:
        cosine_similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    except:
        cosine_similarity=0
        
    return cosine_similarity

In [5]:
path = './Pretrained/glove.6B.300d.txt'
model = pd.read_table(path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
tk = spacy.load('en')


In [6]:
gloveModel = Glove(model,tk,tfidf)

In [7]:
train = pd.read_csv('./Data/simpleFeatures.csv')


In [20]:
avgVec1 = train.q1.apply(lambda x:gloveModel.getAverageVector(str(x)))

In [21]:
avgVec2 = train.q2.apply(lambda x:gloveModel.getAverageVector(str(x)))

In [22]:

avgVec = pd.concat([avgVec1,avgVec2],axis=1)


In [24]:
avgWeighted = avgVec.apply(lambda x:CosineSimilarity(x['q1'],x['q2']),axis=1)

/Users/boom/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
df['gloveAvgVector'] = avgWeighted

In [16]:
avgTFIDF1 = df.q1.apply(lambda x:gloveModel.getAverageTFIDF(str(x)))

In [31]:
avgTFIDF2 = df.q2.apply(lambda x:gloveModel.getAverageTFIDF(str(x)))

In [32]:
avgTFIDF = pd.concat([avgTFIDF1,avgTFIDF2],axis=1)

In [38]:
tfidfCosine = df.apply(lambda x:CosineSimilarity(x['q1'],x['q2']),axis=1)

/Users/boom/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


In [42]:
df['avgTFIDFCosineW1'] = tfidfCosine

In [44]:
glove.to_csv('./Data/gloveCosine.csv')